In [2]:
DF_PATH = "../data/processed/03_X_train_scaled_df.pkl"
DF_PATH1 = "../data/processed/04_X_val_scaled_df.pkl"
DF_PATH2 = "../data/processed/05_y_train_df.pkl"
DF_PATH3 = "../data/processed/06_y_val_df.pkl"
DF_PATH4 = "../data/processed/07_test_df.pkl"
DF_PATH5 = "../data/processed/08_test_scaled_df.pkl"


In [3]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [4]:
# load data
X_train = pd.read_pickle(DF_PATH)
X_val = pd.read_pickle(DF_PATH1)
y_train = pd.read_pickle(DF_PATH2)
y_val = pd.read_pickle(DF_PATH3)
test = pd.read_pickle(DF_PATH5)


print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, test.shape)

(11208, 16) (2803, 16) (11208,) (2803,) (3503, 16)


--------

## fine tune

In [6]:
params = {
    "n_estimators": [10, 20, 30],
    "max_features": ["auto", "sqrt", "log2"],
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 3, 4],
}

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, params, cv=5,scoring="neg_mean_squared_error")
grid_search.fit(X_train, y_train)

d:\Abdelrahmane\project\ml book-ch2_end_to_end_machine_learning_project\e2e_ml_project\.venv\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
d:\Abdelrahmane\project\ml book-ch2_end_to_end_machine_learning_project\e2e_ml_project\.venv\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
d:\Abdelrahmane\project\ml book-ch2_end_to_end_machine_learning_project\e2e_ml_project\.venv\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_feat

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 4],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [10, 20, 30]},
             scoring='neg_mean_squared_error')

In [7]:
grid_search.best_params_

{'max_depth': 4,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 30}

In [8]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=4, max_features='auto', min_samples_split=3,
                      n_estimators=30)

In [9]:
forest_reg = RandomForestRegressor(**grid_search.best_params_)

# fit the model
forest_reg.fit(X_train, y_train)
# make predictions
y_preds = forest_reg.predict(X_val)

d:\Abdelrahmane\project\ml book-ch2_end_to_end_machine_learning_project\e2e_ml_project\.venv\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [10]:
def rmse(y_true, y_preds):
    return np.sqrt(mean_squared_error(y_true, y_preds))


In [11]:
print(f"Validation Score: {forest_reg.score(X_val, y_val)}")
print(f"Validation RMSE: {rmse(y_val, y_preds)}")

Validation Score: 0.625710306951953
Validation RMSE: 62214.92920200517


> The grid search approach is fine when you are exploring relatively few combinations,like in the previous example, but when the hyperparameter search space is large, it is often preferable to use `RandomizedSearchCV()` instead. This class can be used in much the same way as the GridSearchCV class, but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration.

## Evaluate Model on the Test Set

In [12]:
eval_test = forest_reg.predict(test)

In [16]:
# save model
pickle.dump(forest_reg, open("../models/model.pkl", "wb"))

# save predictions
pickle.dump(eval_test, open("../models/predictions.pkl", "wb"))